<a href="https://colab.research.google.com/github/mmsamiei/just-practice-deep/blob/master/Abbas_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torchtext
from torchtext.data import Field
import spacy

from spacy.symbols import ORTH
my_tok = spacy.load('en')

def spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]

QUERY = Field(lower=True, tokenize=spacy_tok)
RESPONSE = Field(lower=True, tokenize=spacy_tok, is_target=True, init_token='<bos>', eos_token='<eos>')

In [0]:
import torch
if torch.cuda.is_available:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [0]:
my_tok.tokenizer.add_special_case("don't", [{ORTH: "do"}, {ORTH: "n't"}])
my_tok.tokenizer.add_special_case("can't", [{ORTH: "can"}, {ORTH: "not"}])

In [0]:
from torchtext.data import TabularDataset

train_dataset = TabularDataset(path="./formatted_movie_lines.txt", format="CSV",
                               fields=[("query", QUERY),("response", RESPONSE)],
                               csv_reader_params={"delimiter":'\t'})

In [6]:
QUERY.build_vocab(train_dataset)
RESPONSE.build_vocab(train_dataset)
print("id of 'film' in query vocab is {}".format(QUERY.vocab.stoi['film']))
print("word of id=33 in query vocab is '{}'".format(QUERY.vocab.itos[33]))
print("len of Query vocab is {}".format(len(QUERY.vocab)))
print("len of Response vocab is {}".format(len(RESPONSE.vocab)))

id of 'film' in query vocab is 1059
word of id=33 in query vocab is 'know'
len of Query vocab is 48505
len of Response vocab is 49036


In [7]:
print("number of rows in train data is {}".format(len(train_dataset)))

number of rows in train data is 221282


In [22]:
from torchtext.data import Dataset

def my_filter_pred(example, limited_word = 3):
  if(len(example.query) < limited_word and len(example.response) < limited_word):
    return True
  else:
    return False

phase_train_dataset = Dataset(examples = train_dataset.examples,
               fields=[("query", QUERY),("response", RESPONSE)],
               filter_pred = my_filter_pred)

print("len of this phase_train_dataset is {}".format(len(phase_train_dataset)))

len of this phase_train_dataset is 1646


In [0]:
from torchtext.data import BucketIterator, interleave_keys

batch_size = 512

train_iterator = BucketIterator(dataset= phase_train_dataset, batch_size=batch_size,
                                device=device,
                                sort_key=lambda x: interleave_keys(len(x.query), len(x.response)),
                                sort = True,
                                shuffle = True,
                                repeat = False)

In [34]:
## test if data loads well?
for a in iter(train_iterator):
  print("response shape : \t",a.response.shape)
  print("query shape :    \t",a.query.shape)

response shape : 	 torch.Size([4, 512])
query shape :    	 torch.Size([2, 512])
response shape : 	 torch.Size([4, 512])
query shape :    	 torch.Size([2, 512])
response shape : 	 torch.Size([4, 512])
query shape :    	 torch.Size([2, 512])
response shape : 	 torch.Size([4, 110])
query shape :    	 torch.Size([2, 110])


In [36]:
num_batch = 0
for batch in (iter(train_iterator)):
  num_batch += 1
print("number of batch is:", num_batch)

number of batch is: 4


In [0]:
import torch.nn as nn
class Bradley(nn.Module):
  def __init__(self, src_voc_sze, trg_voc_sze, hid_sze, num_head, num_enc, num_dec):
    super(Bradley, self).__init__()
    self.hid_sze = hid_sze
    self.src_word_embedding = nn.Embedding(src_voc_sze, self.hid_sze)
    self.trg_word_embedding = nn.Embedding(trg_voc_sze, self.hid_sze)
    self.trg_pos_embedding = nn.Embedding(800, self.hid_sze)
    self.num_head = num_head
    self.transformer = nn.Transformer(self.hid_sze, self.num_head, num_enc, num_dec)
    self.fc = nn.Linear(self.hid_sze, trg_voc_sze)
  
  def forward(self, src, trg):
    #src = [src sent len, batch_size]
    #trg = [trg sent len, batch_size]
    temp_src = self.src_word_embedding(src)
    temp_trg = self.trg_word_embedding(trg)
    trg_sent_len, batch_size = trg.shape[0], trg.shape[1]
    trg_pos = self.trg_pos_embedding(torch.arange(0, trg_sent_len).unsqueeze(0).
                                     repeat(batch_size,1).to(device)).transpose(0,1)
    trg_mask = self._generate_square_subsequent_mask(trg_sent_len)
    temp_trg = temp_trg + trg_pos
    temp = self.transformer(temp_src, temp_trg, tgt_mask=trg_mask)
    return self.fc(temp)
  
  def _generate_square_subsequent_mask(self, sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    mask = mask.to(device)
    return mask


  def greedy_inference_one_sample(self, src, max_len=20):

    self.eval()
    with torch.no_grad():
      #src = [src sent len]
      src_len = src.shape[0]
      src = src.unsqueeze(1)
      #src = [sent_len, 1]
      trg = src.new_full((1,1), RESPONSE.vocab.stoi['<bos>'])
      #trg = [1,1]

      translation_step = 0
      while translation_step < max_len:
        out = self.forward(src, trg)
        out = out[-1,:]
        #out = [batch_size, trg_vocab_size]
        nex = out.argmax(dim=1).unsqueeze(0)
        #nex = [1, 1]
        trg = torch.cat((trg, nex), dim=0)
        translation_step += 1
    return trg



In [0]:
bradley_model = Bradley(src_voc_sze=len(QUERY.vocab), trg_voc_sze=len(RESPONSE.vocab),
                        hid_sze=64, num_head=4,
                        num_enc=4, num_dec=3)
bradley_model = bradley_model.to(device)

In [0]:
from tqdm import tqdm

criterion = nn.CrossEntropyLoss(ignore_index=RESPONSE.vocab.stoi['<pad>'])
lr = 5
optimizer = torch.optim.SGD(bradley_model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
epoch_number = 1

print(len(train_dataset)/batch_size)

for epoch in range(1, epoch_number+1):
  print("epoch ", epoch)
  bradley_model.train()
  for i, batch in enumerate(iter(train_iterator)):
    src = batch.query
    trg = batch.response
    optimizer.zero_grad()
    out = bradley_model(src, trg)
    loss = criterion(out[:-1,:].view(-1, out.shape[2]), trg[1:,:].view(-1))
    if(i%100==0):
      print("{}: {}: {}".format(src.shape[1],i, loss.item()))
    #print('current memory allocated: {}'.format(torch.cuda.memory_allocated() / 1024 ** 2))
    #print(src.shape)
    #print(trg.shape)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(bradley_model.parameters(), 0.5)
    optimizer.step()
    #torch.cuda.empty_cache()
    #del loss
    #del src
    #del trg
    #del out


6915.0625
epoch  1
128: 0: 10.740097045898438
128: 100: 6.313337802886963
128: 200: 6.489552974700928
128: 300: 7.44293212890625
128: 400: 6.569747447967529
64: 500: 6.518950939178467
128: 600: 6.071295738220215
128: 700: 5.481566429138184
64: 800: 6.375940322875977
64: 900: 6.660166263580322
64: 1000: 6.00575590133667
64: 1100: 6.207804203033447
32: 1200: 6.329902648925781
32: 1300: 6.518479347229004
32: 1400: 6.319914817810059
32: 1500: 6.229698657989502
32: 1600: 6.216860771179199
128: 1700: 6.047629356384277
64: 1800: 5.151285171508789
64: 1900: 5.830179691314697
32: 2000: 5.744325160980225
32: 2100: 5.459617614746094
32: 2200: 5.996384143829346
32: 2300: 5.840493202209473
32: 2400: 6.186654090881348
32: 2500: 6.222421646118164
32: 2600: 5.972757816314697
32: 2700: 6.337585926055908
32: 2800: 6.269260406494141
32: 2900: 6.356545448303223
16: 3000: 6.338551998138428
16: 3100: 6.485777854919434
16: 3200: 6.274703025817871
32: 3300: 5.969729423522949
32: 3400: 5.686072826385498
32: 35

In [0]:
source_sentence = train_dataset[50].query
#source_sentence = ['what','is','?']
print(source_sentence)

['is', 'he', 'oily', 'or', 'dry', '?']


In [0]:
x = QUERY.numericalize([source_sentence]).to(device)
x = x.flatten()
print(x.shape)
result = bradley_model.greedy_inference_one_sample(x)

torch.Size([6])


In [0]:
result

tensor([[2],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5]], device='cuda:0')

In [0]:
print(RESPONSE.vocab.itos[2])
print(RESPONSE.vocab.itos[5])
print(RESPONSE.vocab.itos[8])
print(RESPONSE.vocab.itos[3])
print(RESPONSE.vocab.itos[22])
print(RESPONSE.vocab.itos[4])
print(RESPONSE.vocab.itos[66])
print(RESPONSE.vocab.itos[125])


<bos>
,
?
<eos>
!
.
think
little


In [0]:
def batch_index_to_strings(trg):
  # trg = [sent_len, batch_size]
  temp = trg.transpose(0,1)
  for i, row in enumerate(temp):
    print(row)

batch_index_to_strings(result)

tensor([2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       device='cuda:0')


In [0]:
mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

NameError: ignored

In [0]:
mask = (torch.triu(torch.ones(5, 5)) == 1).transpose(0, 1)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

In [0]:
mask